In [ ]:
from pathlib import Path
import warnings
import datetime as dt

from indicators.preparedness import (
    MonthlyResidentsAdm3,
    MonthlyResidentsAdm1,
    SnapshotMostRecentResidentsAdm1,
    SnapshotDiffResidentsAdm1,
    SnapshotTrendsResidentsAdm1,
    SnapshotTrendsResidentsAdm3,
)
from sklearn.exceptions import UndefinedMetricWarning

from utils.aggregate_loader import AggregateSpec, StaticDataSpec, AggregateCollection
from utils.country_config import Country, Language, get_country_config

In [ ]:
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
country = "Haiti"
language = "English"
date = "2016-01-01"
monthly_residents_filename = "aggregates/hti-md-ip/residents"  # in data/shared
admin3_shapefile = "admin3.geojson"  # in data/dagrun
projected_population = "projected_residents.csv"  # in static/data
monthly_residents_dir_name = "indicators"
shared_data_dir = "shared"
static_dir = "static"
dagrun_data_dir = "dagrun"
json_name = "report_components.json"
out_folder = "report_components"
lookback_months = 12

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
language = Language[language]
country = get_country_config(country, language)


aggregate_specs = [
    monthly_res_spec := AggregateSpec(
        name="monthly_residents",
        agg_folder=Path(shared_data_dir) / monthly_residents_dir_name,
        file_glob="residents_indicators_*.csv",
        end_date=dt.datetime.strptime(date, DATE_FORMAT),
        config=country,
        lookback_months=lookback_months,
    ),
    admin3_shapefile := StaticDataSpec(
        name="admin3_shapefile",
        path=Path(dagrun_data_dir) / admin3_shapefile,
        config=country,
    ),
    projected_population := StaticDataSpec(
        name="projected_population",
        path=Path(static_dir) / projected_population,
        config=country,
    ),
]

out_folder = Path(dagrun_data_dir) / "report_components"
json_path = out_folder / json_name

In [ ]:
aggregates = AggregateCollection(aggregate_specs, config=country)

In [ ]:
indicators = [
    MonthlyResidentsAdm3(aggregates),
    MonthlyResidentsAdm1(aggregates),
    SnapshotDiffResidentsAdm1(aggregates),
    SnapshotTrendsResidentsAdm1(aggregates),
    SnapshotTrendsResidentsAdm3(aggregates),
    SnapshotMostRecentResidentsAdm1(aggregates),
]

period_start = dt.date(year=9999, month=1, day=1)
period_end = dt.date(year=1, month=1, day=1)

for indicator in indicators:

    print(f"Indicator: {indicator.__class__.__name__}")
    result = indicator.calculate()
    print(result)
    try:
        if result[country.date_column].max().date() >= period_end:
            period_end = result[country.date_column].max()
        if result[country.date_column].min().date() <= period_start:
            period_start = result[country.date_column].min()
    except (AttributeError, ValueError, KeyError):
        pass

    indicator.save_result(result, "prep_indicators.h5")
    print(f"Date range start: {period_start}")
    print(f"Date range end: {period_end}")

In [ ]:
# Todo: split into two NBs

# Viz

In [ ]:
import os

import matplotlib.pyplot as plt
import geopandas as gpd

from visualisations.preparedness import (
    MapResidentsTrendsNational,
    MapResidentsTrendsByRegion,
    TimeSeriesAggregateResidents,
    TimeSeriesTop3Residents,
    TimeSeriesTop3FluctuatingResidents,
    TablePopVariation,
)

from utils.indicator_loader import IndicatorLoader

In [ ]:
plt.rcParams["font.family"] = "Roboto"
plt.rcParams["font.sans-serif"] = ["Roboto"]
plt.rcParams["font.size"] = 8

In [ ]:
geometry = gpd.read_file(admin3_shapefile.path)
indicators = IndicatorLoader(
    "prep_indicators.h5", config=country, date=date, shapefile=geometry
)

In [ ]:
info_products = [
    MapResidentsTrendsNational(indicators),
    MapResidentsTrendsByRegion(indicators),
    TimeSeriesAggregateResidents(indicators),
    TimeSeriesTop3Residents(indicators),
    TimeSeriesTop3FluctuatingResidents(indicators),
    TablePopVariation(indicators),
]
import json

figure_folder = out_folder / "figures"

out_dict = dict(
    period_start=period_start.strftime("%Y-%m-%d"),
    period_end=period_end.strftime("%Y-%m-%d"),
)

for product in info_products:

    print(f"Figure: {product.__class__.__name__}")
    figures = product.make()

    for figure in figures:
        out_path = figure_folder / figure.filepath
        out_path.parent.mkdir(parents=True, exist_ok=True)
        if figure.figure is not None:
            out_path = out_path.with_suffix(".png")
            plt.savefig(out_path, dpi=300)
            # plt.show()
            plt.close()
        if figure.csv is not None:
            out_path = out_path.with_suffix(".csv")
            figure.csv.to_csv(out_path)

        if out_path.parent.name not in out_dict.keys():
            out_dict[out_path.parent.name] = {}
        out_dict[out_path.parent.name][out_path.stem] = str(out_path)
with open(json_path, "w") as fp:
    json.dump(out_dict, fp)

In [ ]:
# TODO: TimeSeriesAggregateResidents spits out